In [6]:
import kotlinx.coroutines.Dispatchers
import kotlinx.coroutines.async
import kotlinx.coroutines.awaitAll
import kotlinx.coroutines.runBlocking
import com.google.common.hash.Hashing

In [7]:
import io.github.cdimascio.dotenv.Dotenv
import java.nio.file.Paths

val dotenv = Dotenv.load()
val dataDir = dotenv.get("DATA_DIR").let { Paths.get(it).toFile() }.also { it.mkdirs() }
dataDir

/home/j.zeidler@GDAAG.DE/private/ba3/bachelor-thesis/data

In [8]:
import org.bson.BsonDocument
import org.litote.kmongo.*

val client = KMongo.createClient("mongodb://localhost:42692/")
val db = client.getDatabase("s5_snyk_libio")
val mergedVuln = db.getCollection<BsonDocument>("mergedVuln")

In [9]:
import com.mongodb.client.*

private class MongoCursorIterable<T>(private val cursor: MongoCursor<T>) : MongoCursor<T> by cursor, Iterable<T> {

    override fun iterator(): Iterator<T> = cursor
}

private fun <T> MongoIterable<T>.kCursor(): MongoCursorIterable<T> = MongoCursorIterable(iterator())

fun <T, R> MongoIterable<T>.useCursor(block: (Iterable<T>) -> R): R {
    return kCursor().use(block)
}

In [10]:
val mergedVulnGavs = mergedVuln.find().useCursor { blk ->
    blk.map {
        it["vuln_gav"]!!.asString().value
    }
}.toSet()
mergedVulnGavs.count()

3809

In [11]:
val jarsDir = dataDir.resolve("interim/jars").also { it.mkdirs() }

fun gavToJarUrl(packageGAV: String): String
{
    val parts = packageGAV.split(":")
    return parts[0].replace('.', '/') + "/" + parts[1] + "/" + parts[2] + "/" + parts[1] + "-" + parts[2] + ".jar"
}

In [12]:
val gavToJar = mergedVulnGavs
    .map {
        val jarPath = jarsDir.resolve(gavToJarUrl(it))
        if (!jarPath.isFile()) null
        else it to jarPath
    }
    .filterNotNull()
    .toMap()
gavToJar.count()  // 3856

3809

In [13]:
import java.io.FileNotFoundException
import java.util.zip.ZipFile

fun classExistsInJar(jar: File, classFilePath: String): Boolean {
    if (!jar.isFile()) {
        throw FileNotFoundException("jar file not found: ${jar.absolutePath}")
    }
    
    val jarAsZip = ZipFile(jar)
    val entry = jarAsZip.getEntry(classFilePath)
    if (entry == null)
        return false
    
    return true
}

In [14]:
import org.bson.BsonArray

val filteredItems = mergedVuln.find().useCursor { blk ->
    blk.filter {
        gavToJar.containsKey(it["vuln_gav"]!!.asString().value)
    }.map {
        val jar = gavToJar[it["vuln_gav"]!!.asString().value]!!
        val modifiedClassesInJar = it["vuln_classes"]!!.asArray()
        for (cls in modifiedClassesInJar)
            if (!classExistsInJar(jar,cls.asString().value.replace('.', '/') + ".class")) print(cls)
    }.filterNotNull()
}